In [3]:
%matplotlib inline

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings

In [5]:
warnings.filterwarnings("ignore")

## 1. 데이터프레임 만들기 (csv 파일로 만들기)

In [6]:
testData = [
{'create_dt':'2020-09-18', 'contents':'손흥민 의조선수 국대 선수님들 파팅요!'},
{'create_dt':'2020-09-01', 'contents':'손흥민 만두 먹고 왔다. test contents'},
{'create_dt':'2020-09-02', 'contents':'손흥민은 만두를 싫어 한다.'},
{'create_dt':'2020-09-03', 'contents':'손흥민은 만두를 만드는것을 좋아한다.'},
{'create_dt':'2020-09-04', 'contents':'TS샴푸! 손흥민 광고모델! 손흥!'}
]


columns = ['생성일', '본문']
df = pd.DataFrame(columns=columns)

# for i in range(1, 100):
#     print("Get data", str(i/500*100), "% complete...")
for data0 in testData:
    data_dt = data0['create_dt']
    data_ct = data0['contents']
    row = [data_dt, data_ct]
    series = pd.Series(row, index=df.columns)
    df = df.append(series, ignore_index=True)
# print("Get data 100 % complete...")

In [7]:
# csv파일로 저장
df.to_csv('tweet_temp.csv', index=False)

## 2. 키워드 추출

#### [2-1) 텍스트 데이터 전처리]

In [8]:
# csv파일 불러오기
df = pd.read_csv('tweet_temp.csv')
df.head()

,생성일,본문
0,2020-09-18,손흥민 의조선수 국대 선수님들 파팅요!
1,2020-09-01,손흥민 만두 먹고 왔다. test contents
2,2020-09-02,손흥민은 만두를 싫어 한다.
3,2020-09-03,손흥민은 만두를 만드는것을 좋아한다.
4,2020-09-04,TS샴푸! 손흥민 광고모델! 손흥!


In [9]:
import re

# 텍스트 정제 함수 : 할글 이외의 문자는 전부 제거
def text_cleaning(text):
    hangul = re.compile('[^ㄱ-ㅣ가-힣]+') #한글의 정규표현식을 나타냅니다.
    result = hangul.sub('', text)
    return result

In [10]:
# 'tweet_text' 피처에 이를 적용합니다.
df['ko_text'] = df['본문'].apply(lambda x: text_cleaning(x))
df.head()

,생성일,본문,ko_text
0,2020-09-18,손흥민 의조선수 국대 선수님들 파팅요!,손흥민의조선수국대선수님들파팅요
1,2020-09-01,손흥민 만두 먹고 왔다. test contents,손흥민만두먹고왔다
2,2020-09-02,손흥민은 만두를 싫어 한다.,손흥민은만두를싫어한다
3,2020-09-03,손흥민은 만두를 만드는것을 좋아한다.,손흥민은만두를만드는것을좋아한다
4,2020-09-04,TS샴푸! 손흥민 광고모델! 손흥!,샴푸손흥민광고모델손흥


#### [2-2) KoNLPy를 이용한 키워드 추출 (형태소 분석기)]
#### 한국어 약식 불용어 사전 사용 (http://ranks.nl/stopwords/korean)

In [11]:

from konlpy.tag import Okt
from collections import Counter

korean_stopwords_path = './korean_stopwords.txt'
with open(korean_stopwords_path, encoding='utf8') as f:
    stopwords = f.readlines()
stopwords = [x.strip() for x in stopwords]

def get_nouns(x):
    nouns_tagger = Okt()
    nouns = nouns_tagger.nouns(x)

    #한글자 키워드를 제거합니다.
    nouns = [noun for noun in nouns if len(noun) > 1]

    #불용어를 제거합니다.
    nouns = [noun for noun in nouns if noun not in stopwords]

    return nouns


In [12]:
df['nouns'] = df['ko_text'].apply(lambda x: get_nouns(x))
print(df.shape)
df.head()

(5, 4)


,생성일,본문,ko_text,nouns
0,2020-09-18,손흥민 의조선수 국대 선수님들 파팅요!,손흥민의조선수국대선수님들파팅요,"[손흥민, 조선, 수국, 선수, 파팅]"
1,2020-09-01,손흥민 만두 먹고 왔다. test contents,손흥민만두먹고왔다,[손흥민]
2,2020-09-02,손흥민은 만두를 싫어 한다.,손흥민은만두를싫어한다,"[손흥민, 만두]"
3,2020-09-03,손흥민은 만두를 만드는것을 좋아한다.,손흥민은만두를만드는것을좋아한다,"[손흥민, 만두]"
4,2020-09-04,TS샴푸! 손흥민 광고모델! 손흥!,샴푸손흥민광고모델손흥,"[샴푸, 손흥민, 광고, 모델, 손흥]"


## 3. 연관 키워드 추출하기

!pip install apriori apyori

### [연관 분석 연습]

In [13]:
from apyori import apriori

# 1) 트랜잭션 데이터를 생성
transactions = [
    ['손흥민','시소코'],
    ['손흥민','케인'],
    ['손흥민','케인','포체티노']
]

# 2) 연관 분석을 수행합니다.
results = list(apriori(transactions))
for result in results:
    print(result)

RelationRecord(items=frozenset({'손흥민'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'손흥민'}), confidence=1.0, lift=1.0)])
RelationRecord(items=frozenset({'시소코'}), support=0.3333333333333333, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'시소코'}), confidence=0.3333333333333333, lift=1.0)])
RelationRecord(items=frozenset({'케인'}), support=0.6666666666666666, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'케인'}), confidence=0.6666666666666666, lift=1.0)])
RelationRecord(items=frozenset({'포체티노'}), support=0.3333333333333333, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'포체티노'}), confidence=0.3333333333333333, lift=1.0)])
RelationRecord(items=frozenset({'시소코', '손흥민'}), support=0.3333333333333333, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'시소코', '손흥민'}), confidence=0.3333333333333333, lift=1.0), Or

In [14]:
# 3) 연관 분석 필터기능 사용해보기.
# 지지도 0.5, 신뢰도 0.6, 향상도 1.0 이상이면서 (손흥민, 케인) 처헐 규칙의 크기가 2 이하의 규칙을 추출
results = list(apriori(transactions,
                        min_support=0.5,
                        min_confidence=0.6,
                        min_lift=1.0,
                        max_length=2))

for result in results:
    print(result)

RelationRecord(items=frozenset({'손흥민'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'손흥민'}), confidence=1.0, lift=1.0)])
RelationRecord(items=frozenset({'케인'}), support=0.6666666666666666, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'케인'}), confidence=0.6666666666666666, lift=1.0)])
RelationRecord(items=frozenset({'케인', '손흥민'}), support=0.6666666666666666, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'손흥민', '케인'}), confidence=0.6666666666666666, lift=1.0), OrderedStatistic(items_base=frozenset({'손흥민'}), items_add=frozenset({'케인'}), confidence=0.6666666666666666, lift=1.0), OrderedStatistic(items_base=frozenset({'케인'}), items_add=frozenset({'손흥민'}), confidence=1.0, lift=1.0)])


### [트위터 연관 키워드 분석]

In [15]:
transactions = df['nouns'].tolist()
transactions = [transaction for transaction in transactions if transaction] # 공백 문자열을 방지합니다.and
print(transactions)

[['손흥민', '조선', '수국', '선수', '파팅'], ['손흥민'], ['손흥민', '만두'], ['손흥민', '만두'], ['샴푸', '손흥민', '광고', '모델', '손흥']]


In [53]:

ress = list(apriori(transactions))

# , min_support=0.1, min_confidence=0.2, min_lift=5, max_length=2))
print(ress)

#테스트 진행중.

[RelationRecord(items=frozenset({'광고'}), support=0.2, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'광고'}), confidence=0.2, lift=1.0)]), RelationRecord(items=frozenset({'만두'}), support=0.4, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'만두'}), confidence=0.4, lift=1.0)]), RelationRecord(items=frozenset({'모델'}), support=0.2, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'모델'}), confidence=0.2, lift=1.0)]), RelationRecord(items=frozenset({'샴푸'}), support=0.2, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'샴푸'}), confidence=0.2, lift=1.0)]), RelationRecord(items=frozenset({'선수'}), support=0.2, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'선수'}), confidence=0.2, lift=1.0)]), RelationRecord(items=frozenset({'손흥'}), support=0.2, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'손흥'

In [54]:

columns = ['소스/타겟', 'support(지지도)', 'confidence(신뢰도)', 'lift(향상도)']
network_dl = pd.DataFrame(columns=columns)



for result in ress :
    # print(items[0], items[1],result.support, result.ordered_statistics[0].confidence, result.ordered_statistics[0].lift)
    if len(result.items) == 2 :
        items = [x for x in result.items]
        row = [items, result.support, result.ordered_statistics[0].confidence, result.ordered_statistics[0].lift]
        series = pd.Series(row, index=network_dl.columns)
        network_dl = network_dl.append(series, ignore_index=True)

#series.append() = [network_dl['소스/타겟'].apply(lambda x : '손흥민' in x)]

# network_dl['테스트'] = network_dl['소스/타겟']
a = network_dl[network_dl['소스/타겟'].apply(lambda x : '손흥민' in x)]

# type(a)

# print(a)

# network_dl.head()
a.head()

,소스/타겟,support(지지도),confidence(신뢰도),lift(향상도)
3,"[광고, 손흥민]",0.2,0.2,1.0
4,"[만두, 손흥민]",0.4,0.4,1.0
7,"[모델, 손흥민]",0.2,0.2,1.0
9,"[샴푸, 손흥민]",0.2,0.2,1.0
10,"[선수, 손흥민]",0.2,0.2,1.0


In [55]:
test_list = []
for tt in a['소스/타겟'] :
    for xx in tt :
        if(xx != '손흥민') :
            test_list.append(xx)


print(test_list)

['광고', '만두', '모델', '샴푸', '선수', '손흥', '수국', '조선', '파팅']


In [56]:
a['연관키워드 추출'] = a['소스/타겟'].apply(lambda x: [y for y in x if y != '손흥민'])

a

,소스/타겟,support(지지도),confidence(신뢰도),lift(향상도),연관키워드 추출
3,"[광고, 손흥민]",0.2,0.2,1.0,[광고]
4,"[만두, 손흥민]",0.4,0.4,1.0,[만두]
7,"[모델, 손흥민]",0.2,0.2,1.0,[모델]
9,"[샴푸, 손흥민]",0.2,0.2,1.0,[샴푸]
10,"[선수, 손흥민]",0.2,0.2,1.0,[선수]
14,"[손흥, 손흥민]",0.2,0.2,1.0,[손흥]
15,"[수국, 손흥민]",0.2,0.2,1.0,[수국]
16,"[조선, 손흥민]",0.2,0.2,1.0,[조선]
17,"[파팅, 손흥민]",0.2,0.2,1.0,[파팅]
